In [1]:
abs_path = 'C:/Users/ollieosunkunle/Google Drive/Work, career and education/Programming_data_science/UFC fight predictor/Version 2/'

In [2]:
test_file = abs_path + "test_data/sep_test.csv"

In [3]:
#Import various useful libraries
import pandas as pd
import numpy as np

import datetime as dt
import datetime

from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import seaborn as sns

from fuzzywuzzy import process

C:\Users\ollieosunkunle\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
sql_con = "sqlite:///../database/ufc_data.db"
con = create_engine(sql_con)

In [5]:
test_df = pd.read_csv(test_file, encoding='latin-1', sep='\t',usecols = ['fighter_1','fighter_2'])

In [6]:
train_df = pd.read_sql("SELECT * FROM train_df_v3", con)

In [7]:
# Find out which fighter names from test_set are missing and identify the closest matches
display('Original fighter list',test_df)

train_fighters = train_df['fighter_1'].unique() #Create a list of unique train_fighters
test_fighters = test_df['fighter_1'].append(test_df['fighter_2']) #Create a list of unique test_fighters
missing_fighters = test_fighters[~test_fighters.isin(train_fighters)].reset_index(drop=True) #Find the missing fighters in the test_dataset
closest_matches = missing_fighters.apply(lambda x: process.extract(x,train_fighters,limit=1)[0])
fighter_lu = pd.concat([missing_fighters,closest_matches],axis=1)
fighter_lu.columns = ['original_name','new_name']
threshold = 85 #Set the matching threshold to replace an old fighter's name with a new one
fighter_lu['new_name'] = fighter_lu['new_name'].apply(lambda x: x[0] if x[1] > threshold else np.nan) #Set to nan names that don't meet threshold

#display('Missing fighters and lookups',fighter_lu)

# Replace the names in the original test dataset
replacement_dict = fighter_lu.set_index('original_name').to_dict()['new_name']
for f in ['fighter_1','fighter_2']:
    test_df[f] = test_df[f].replace(replacement_dict, value=None)

#display('Merged and replaced missing fighters',test_df)
test_df = test_df.dropna()

display('Final fighter list',test_df)
test_df = test_df.reset_index(drop=True)

'Original fighter list'

,fighter_1,fighter_2
0,Michelle Waterson,Angela Hill
1,Ottman Azaitar,Khama Worthy
2,Roxanne Modafferi,Andrea Lee
3,Ed Herman,Mike Rodríguez
4,Bobby Green,Alan Patrick
5,Billy Quarantillo,Kyle Nelson
6,Sijara Eubanks,Julia Avila
7,Kevin Croom,Roosevelt Roberts
8,Alexander Romanov,Roque Martinez
9,Jalin Turner,Brok Weaver


'Final fighter list'

,fighter_1,fighter_2
0,Michelle Waterson,Angela Hill
1,Ottman Azaitar,Khama Worthy
2,Roxanne Modafferi,Andrea Lee
3,Ed Herman,Mike Rodriguez
4,Bobby Green,Alan Patrick
5,Billy Quarantillo,Kyle Nelson
6,Sijara Eubanks,Julia Avila
9,Jalin Turner,Brok Weaver
10,Bryan Barberena,Anthony Ivy
11,Sabina Mazo,Justine Kish


In [8]:
# Define the difference dictionary (note - this is copied from the create_training_df model)
# Subset the train_df for f1_features - dictionary keys
# Merge f1 features to fighter_1
# Rename the subset dataset to f2
# Merge f2 features to fighter_2
# Calculate the difference features